# 基于Qianfan和BES的RAG

这个笔记本是使用百度千帆平台和百度ElasticSearch实现的检索增强生成（RAG），其中原始数据位于BOS上。

## 百度千帆
百度AI云千帆平台是一个为企业开发者提供一站式大型模型开发和服务运营平台。千帆不仅提供了包括文心一言（ERNIE-Bot）模型和第三方开源模型在内的各种模型，还提供了各种AI开发工具和完整的开发环境，方便客户轻松使用和开发大型模型应用。

## 百度ElasticSearch
[百度云向量搜索](https://cloud.baidu.com/doc/BES/index.html?from=productToDoc)是一个完全托管的、企业级的分布式搜索和分析服务，100%兼容开源。百度云向量搜索为结构化/非结构化数据提供低成本、高性能和可靠的检索和分析平台级产品服务。作为一个向量数据库，它支持多种索引类型和相似度距离方法。

## 安装和设置

In [ ]:
# 安装必要的依赖库
!pip install qianfan
!pip install bce-python-sdk
!pip install elasticsearch==7.11.0
!pip install sentence-transformers

## 导入模块

In [ ]:
import sentence_transformers
from baidubce.auth.bce_credentials import BceCredentials
from baidubce.bce_client_configuration import BceClientConfiguration
from langchain.chains.retrieval_qa import RetrievalQA
from langchain_community.document_loaders.baiducloud_bos_directory import (
    BaiduBOSDirectoryLoader,
)
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.llms.baidu_qianfan_endpoint import QianfanLLMEndpoint
from langchain_community.vectorstores import BESVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

## 文档加载

In [ ]:
bos_host = "your bos eddpoint"
access_key_id = "your bos access ak"
secret_access_key = "your bos access sk"

# 创建BceClientConfiguration
config = BceClientConfiguration(
    credentials=BceCredentials(access_key_id, secret_access_key), endpoint=bos_host
)

loader = BaiduBOSDirectoryLoader(conf=config, bucket="llm-test", prefix="llm/")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
split_docs = text_splitter.split_documents(documents)

## 嵌入和向量存储

In [ ]:

# 使用HuggingFaceEmbeddings模型创建词嵌入
embeddings = HuggingFaceEmbeddings(model_name="shibing624/text2vec-base-chinese")
# 使用SentenceTransformer客户端加载模型
embeddings.client = sentence_transformers.SentenceTransformer(embeddings.model_name)

# 从文档中创建BESVectorStore
db = BESVectorStore.from_documents(
    documents=split_docs,  # 输入的文档列表
    embedding=embeddings,  # 使用的词嵌入模型
    bes_url="your bes url",  # BES的URL
    index_name="test-index",  # 索引名称
    vector_query_field="vector",  # 向量查询字段
)

# 刷新索引
db.client.indices.refresh(index="test-index")
# 将BESVectorStore转换为检索器
retriever = db.as_retriever()

## QA 检索器

In [ ]:

# 创建 QianfanLLMEndpoint 对象，指定模型为 "ERNIE-Bot"，并提供 Qianfan 的 AK 和 SK
llm = QianfanLLMEndpoint(
    model="ERNIE-Bot",
    qianfan_ak="your qianfan ak",
    qianfan_sk="your qianfan sk",
    streaming=True,
)

# 创建 RetrievalQA 对象，指定链式类型为 "refine"，并提供检索器和返回源文档的参数
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="refine", retriever=retriever, return_source_documents=True
)

# 设置查询字符串
query = "什么是张量?"

# 运行查询并打印结果
print(qa.run(query))

以上代码使用了 Qianfan SDK 来实现一个问答系统。首先，创建了一个 QianfanLLMEndpoint 对象，指定了模型为 "ERNIE-Bot"，并提供了 Qianfan 的 AK 和 SK。然后，创建了一个 RetrievalQA 对象，指定了链式类型为 "refine"，并提供了检索器和返回源文档的参数。接下来，设置了查询字符串为 "什么是张量?"，最后运行查询并打印结果。

A tensor is a mathematical concept used to represent multi-dimensional data. It is an array that can hold multiple values and can be a scalar, vector, matrix, etc. In the field of deep learning and artificial intelligence, tensors are commonly used to represent inputs, outputs, and weights of neural networks.